привет. начну с датасета. не буду ничего выдумывать, а просто сделаю первое задание со звездочкой, а именно: реализую автоматический сбор данных с помощью API. В качестве источника для сбора данных я взяла сервис rawg, в котором зарегистрировалась и получила свой API ключ, который далее использовала. Собрала все в датафрейм и готово. Единственное что - это то, что в данном источнике не было данных об описании и конкурентах.

api_key = '130f7d046ec84b01b9b52e9d75481a8f'

In [6]:
import requests
import pandas as pd

def fetch_all_games(api_key):
    url = "https://api.rawg.io/api/games"
    games_list = []
    
    page = 1
    while True:
        params = {
            'key': api_key,
            'page_size': 10,  
            'page': page 
        }
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Ошибка запроса: статус код {response.status_code}")
            break
        
        try:
            data = response.json()
        except ValueError as e:
            print("Ошибка при декодировании JSON:", e)
            break
        
        games_data = data['results']
        if not games_data:
            print("Игры на странице закончились.")
            break
        
        for game in games_data:
            metacritic_score = game.get('metacritic', 0) or 0
            game_info = {
                "Название игры": game['name'],
                "Жанры": ', '.join([genre['name'] for genre in game['genres']]),
                "Описание": game.get('description_raw', 'Нет описания'),
                "Целевая аудитория": "Все возрастные группы" if game['esrb_rating'] is None else game['esrb_rating']['name'],
                "Прибыльность": "Высокая" if metacritic_score > 75 else "Средняя" if metacritic_score > 50 else "Низкая",
                "Конкуренты": "Исследование требуется"
            }
            games_list.append(game_info)
        
        page += 1  

    df = pd.DataFrame(games_list)
    return df

api_key = '130f7d046ec84b01b9b52e9d75481a8f'
df_games = fetch_all_games(api_key)



Ошибка запроса: статус код 502


In [7]:
df_games

,Название игры,Жанры,Описание,Целевая аудитория,Прибыльность,Конкуренты
0,Grand Theft Auto V,Action,Нет описания,Mature,Высокая,Исследование требуется
1,The Witcher 3: Wild Hunt,"Action, RPG",Нет описания,Mature,Высокая,Исследование требуется
2,Portal 2,"Shooter, Puzzle",Нет описания,Everyone 10+,Высокая,Исследование требуется
3,Counter-Strike: Global Offensive,Shooter,Нет описания,Mature,Высокая,Исследование требуется
4,Tomb Raider (2013),"Action, Fighting",Нет описания,Mature,Высокая,Исследование требуется
...,...,...,...,...,...,...
12265,Kick Ass Commandos,"Action, Adventure, Strategy, Arcade, Casual, I...",Нет описания,Все возрастные группы,Низкая,Исследование требуется
12266,Dragon Blood,"Action, Adventure, RPG, Massively Multiplayer",Нет описания,Все возрастные группы,Низкая,Исследование требуется
12267,FIVE: Guardians of David,"Action, Adventure, RPG",Нет описания,Все возрастные группы,Средняя,Исследование требуется
12268,Nyheim,Strategy,Нет описания,Все возрастные группы,Низкая,Исследование требуется


In [8]:
df_games.to_csv('games_dataset.csv', index=False)

print("Данные успешно сохранены в файл 'games_dataset.csv'")

Данные успешно сохранены в файл 'games_dataset.csv'


In [9]:
df_games.head(10)

,Название игры,Жанры,Описание,Целевая аудитория,Прибыльность,Конкуренты
0,Grand Theft Auto V,Action,Нет описания,Mature,Высокая,Исследование требуется
1,The Witcher 3: Wild Hunt,"Action, RPG",Нет описания,Mature,Высокая,Исследование требуется
2,Portal 2,"Shooter, Puzzle",Нет описания,Everyone 10+,Высокая,Исследование требуется
3,Counter-Strike: Global Offensive,Shooter,Нет описания,Mature,Высокая,Исследование требуется
4,Tomb Raider (2013),"Action, Fighting",Нет описания,Mature,Высокая,Исследование требуется
5,Portal,"Action, Puzzle",Нет описания,Teen,Высокая,Исследование требуется
6,Left 4 Dead 2,"Action, Shooter",Нет описания,Mature,Высокая,Исследование требуется
7,The Elder Scrolls V: Skyrim,"Action, RPG",Нет описания,Mature,Высокая,Исследование требуется
8,Red Dead Redemption 2,Action,Нет описания,Mature,Высокая,Исследование требуется
9,BioShock Infinite,"Action, Shooter",Нет описания,Mature,Высокая,Исследование требуется


In [10]:
df_games.value_counts('Прибыльность')

Прибыльность
Низкая     7788
Высокая    2336
Средняя    2146
Name: count, dtype: int64

In [11]:
df_games.shape

(12270, 6)

Итого, мы имеем 7580 игр и их соответствующие жанры, целевую аудиторию и прибыльность

Воспользуемся уже проверенным способом и найдем описание из другой базы данных

In [22]:
import requests
import asyncio
import aiohttp
import pandas as pd
from aiohttp import ClientSession
from aiohttp_retry import RetryClient, ExponentialRetry
import nest_asyncio

nest_asyncio.apply()

client_id = 'yijhg4l0zfomvpcc2v72eezcuf2rdb'
client_secret = 'sh4f1y0p96oluvjsom0mffszdubi5c'

def get_access_token(client_id, client_secret):
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.post(url, params=params)
    access_token = response.json().get('access_token')
    return access_token

access_token = get_access_token(client_id, client_secret)
print("Access Token:", access_token)

async def fetch_game_description(session, game_name):
    url = "https://api.igdb.com/v4/games"
    headers = {
        'Client-ID': client_id,
        'Authorization': f'Bearer {access_token}',
    }
    data = f'search "{game_name}"; fields name, summary;'

    async with session.post(url, headers=headers, data=data) as response:
        if response.status == 200:
            games = await response.json()
            if games:
                return games[0].get('summary', 'Нет описания')


        return 'Нет описания'

async def fetch_all_descriptions(df):
    retry_options = ExponentialRetry(attempts=3)
    async with RetryClient(raise_for_status=False, retry_options=retry_options) as session:
        tasks = []
        cache = {}
        for index, row in df.iterrows():
            game_name = row["Название игры"]
            if game_name in cache:
                df.at[index, "Описание"] = cache[game_name]
            else:
                tasks.append(fetch_game_description(session, game_name))

        results = await asyncio.gather(*tasks)

        for index, description in enumerate(results):
            game_name = df.at[index, "Название игры"]
            df.at[index, "Описание"] = description
            cache[game_name] = description

asyncio.run(fetch_all_descriptions(df_games))


df_games.to_csv('games_dataset.csv', index=False)
print("Данные успешно сохранены в файл 'games_dataset.csv'")


Access Token: 4gp0objshl4gt1khjlydn23635e3dh


/opt/anaconda3/lib/python3.11/asyncio/futures.py:233: RuntimeWarning: coroutine 'fetch_all_descriptions' was never awaited
  context = contextvars.copy_context()


Данные успешно сохранены в файл 'games_dataset.csv'


In [30]:
no_description_count = df_games[df_games['Описание'] == 'Нет описания'].shape[0]

print(f"Количество строк с 'Нет описания': {no_description_count}")

Количество строк с 'Нет описания': 12025


In [34]:
import requests
import asyncio
import aiohttp
import pandas as pd
from aiohttp import ClientSession
from aiohttp_retry import RetryClient, ExponentialRetry
import nest_asyncio
import logging

nest_asyncio.apply()

client_id = 'yijhg4l0zfomvpcc2v72eezcuf2rdb'
client_secret = 'sh4f1y0p96oluvjsom0mffszdubi5c'

def get_access_token(client_id, client_secret):
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.post(url, params=params)
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        return access_token
    else:
        raise Exception(f"Failed to get access token: {response.status_code}, {response.text}")

access_token = get_access_token(client_id, client_secret)
logging.info(f"Access Token: {access_token}")

async def fetch_game_description(session, game_name):
    url = "https://api.igdb.com/v4/games"
    headers = {
        'Client-ID': client_id,
        'Authorization': f'Bearer {access_token}',
    }
    data = f'fields name, summary; where name ~ "{game_name}";'

    async with session.post(url, headers=headers, data=data) as response:
        if response.status == 200:
            games = await response.json()
            if games:
                return games[0].get('summary', 'Нет описания')
        return 'Нет описания'

async def fetch_all_descriptions(df):
    retry_options = ExponentialRetry(attempts=3)
    async with RetryClient(raise_for_status=False, retry_options=retry_options) as session:
        tasks = []
        cache = {}
        for index, row in df.iterrows():
            game_name = row["Название игры"]
            if game_name in cache:
                df.at[index, "Описание"] = cache[game_name]
            else:
                tasks.append(fetch_game_description(session, game_name))

        results = await asyncio.gather(*tasks)

        for index, description in enumerate(results):
            game_name = df.at[index, "Название игры"]
            df.at[index, "Описание"] = description
            cache[game_name] = description

df_games["Название игры"] = df_games["Название игры"].str.strip().str.lower()


asyncio.run(fetch_all_descriptions(df_games))





INFO:root:Access Token: uncwknvikm8wn3put1mzeidd4iwcyb


In [36]:
no_description_count = df_games[df_games['Описание'] == 'Нет описания'].shape[0]

print(f"Количество строк с 'Нет описания': {no_description_count}")

Количество строк с 'Нет описания': 12018


Итого, нам удалось найти 12270-12018 = 252 описания ((( Грустно, но как есть. Теперь постараемся найти конкурентов из имеющихся данных 

In [37]:
df_games.to_csv('df_games.csv')

Конкурентов вычислим самостоятельно:

Функция fetch_game_data: Эта функция использует IGDB API для поиска игры по имени и возвращает данные о игре, включая жанры и платформы.

Функция fetch_competitors: Эта функция использует данные о жанрах и платформах для поиска других игр, которые можно считать конкурентами.

Заполнение DataFrame: Проходим по каждой строке DataFrame с помощью метода iterrows и обновляем столбцы "Описание" и "Конкуренты" вызовами функций fetch_game_data и fetch_competitors.

In [14]:
import requests
import asyncio
import aiohttp
import pandas as pd
from aiohttp_retry import RetryClient, ExponentialRetry
import nest_asyncio
import logging

nest_asyncio.apply()


client_id = 'yijhg4l0zfomvpcc2v72eezcuf2rdb'
client_secret = 'sh4f1y0p96oluvjsom0mffszdubi5c'

def get_access_token(client_id, client_secret):
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.post(url, params=params)
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        return access_token
    else:
        raise Exception(f"Failed to get access token: {response.status_code}, {response.text}")

access_token = get_access_token(client_id, client_secret)
print("Access Token:", access_token)

async def fetch_game_data(session, game_name):
    url = "https://api.igdb.com/v4/games"
    headers = {
        'Client-ID': client_id,
        'Authorization': f'Bearer {access_token}',
    }
    data = f'fields id, name, summary, genres, platforms; where name ~ "{game_name}";'

    async with session.post(url, headers=headers, data=data) as response:
        if response.status == 200:
            games = await response.json()
            if games:
                return games[0]
        return None

async def fetch_competitors(session, game_data):
    if game_data is None:
        return "Нет данных"

    genres = game_data.get('genres', [])
    platforms = game_data.get('platforms', [])

    competitors = []
    
    url = "https://api.igdb.com/v4/games"
    headers = {
        'Client-ID': client_id,
        'Authorization': f'Bearer {access_token}',
    }
    data = f'fields name; where genres = ({",".join(map(str, genres))}) & platforms = ({",".join(map(str, platforms))}) & id != {game_data["id"]}; limit 5;'

    async with session.post(url, headers=headers, data=data) as response:
        if response.status == 200:
            games = await response.json()
            for game in games:
                competitors.append(game['name'])

    return ", ".join(competitors) if competitors else "Нет конкурентов"

async def fetch_all_data(df):
    retry_options = ExponentialRetry(attempts=3)
    async with RetryClient(raise_for_status=False, retry_options=retry_options) as session:
        tasks = []
        for index, row in df.iterrows():
            game_name = row["Название игры"]
            tasks.append(fetch_game_data(session, game_name))
        
        game_data_list = await asyncio.gather(*tasks)
        
        competitor_tasks = []
        for game_data in game_data_list:
            competitor_tasks.append(fetch_competitors(session, game_data))
        
        competitors_list = await asyncio.gather(*competitor_tasks)
        
        for index, competitors in enumerate(competitors_list):
            df.at[index, "Конкуренты"] = competitors



df_games = pd.read_csv('df_games.csv')
asyncio.run(fetch_all_data(df_games))
df_games.to_csv('games_dataset.csv', index=False)
print("Данные успешно сохранены в файл 'games_dataset.csv'")


Access Token: isjbrlwqfocmjvehd3mcdkws88hvux
Данные успешно сохранены в файл 'games_dataset.csv'


In [15]:
df_games

,Unnamed: 0,Название игры,Жанры,Описание,Целевая аудитория,Прибыльность,Конкуренты
0,0,grand theft auto v,Action,This bundle contains the original version of G...,Mature,Высокая,Нет данных
1,1,the witcher 3: wild hunt,"Action, RPG","RPG and sequel to The Witcher 2 (2011), The Wi...",Mature,Высокая,Нет конкурентов
2,2,portal 2,"Shooter, Puzzle","Sequel to the acclaimed Portal (2007), Portal ...",Everyone 10+,Высокая,"Soldier Front 2, Meat & Greed, Shadow Wolf Mys..."
3,3,counter-strike: global offensive,Shooter,Counter-Strike: Global Offensive expands upon ...,Mature,Высокая,"Soldier Front 2, Mobile Suit Gundam Battle Ope..."
4,4,tomb raider (2013),"Action, Fighting",Нет описания,Mature,Высокая,Нет данных
...,...,...,...,...,...,...,...
12265,12265,kick ass commandos,"Action, Adventure, Strategy, Arcade, Casual, I...",Нет описания,Все возрастные группы,Низкая,Нет данных
12266,12266,dragon blood,"Action, Adventure, RPG, Massively Multiplayer",Нет описания,Все возрастные группы,Низкая,Нет данных
12267,12267,five: guardians of david,"Action, Adventure, RPG",FIVE: Guardians of David is a Diablo-style act...,Все возрастные группы,Средняя,Нет данных
12268,12268,nyheim,Strategy,Нет описания,Все возрастные группы,Низкая,Нет данных


In [17]:
df_games['Конкуренты'].value_counts

<bound method IndexOpsMixin.value_counts of 0                                               Нет данных
1                                          Нет конкурентов
2        Soldier Front 2, Meat & Greed, Shadow Wolf Mys...
3        Soldier Front 2, Mobile Suit Gundam Battle Ope...
4                                               Нет данных
                               ...                        
12265                                           Нет данных
12266                                           Нет данных
12267                                           Нет данных
12268                                           Нет данных
12269                                           Нет данных
Name: Конкуренты, Length: 12270, dtype: object>

In [18]:
no_conc_count = df_games[df_games['Конкуренты'] == 'Нет данных'].shape[0]

print(f"Количество строк с 'Нет данных': {no_conc_count}")

Количество строк с 'Нет данных': 11976


Видим, что конкурентов удалось вычислить для 12270 - 11976 = 294 игр 

In [19]:
df_games = df_games.drop(columns=['Unnamed: 0'])

In [20]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12270 entries, 0 to 12269
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Название игры      12270 non-null  object
 1   Жанры              12122 non-null  object
 2   Описание           12270 non-null  object
 3   Целевая аудитория  12270 non-null  object
 4   Прибыльность       12270 non-null  object
 5   Конкуренты         12270 non-null  object
dtypes: object(6)
memory usage: 575.3+ KB


In [22]:
df_games.to_csv('full_games_dataset.csv', index=False)

In [23]:
df_games

,Название игры,Жанры,Описание,Целевая аудитория,Прибыльность,Конкуренты
0,grand theft auto v,Action,This bundle contains the original version of G...,Mature,Высокая,Нет данных
1,the witcher 3: wild hunt,"Action, RPG","RPG and sequel to The Witcher 2 (2011), The Wi...",Mature,Высокая,Нет конкурентов
2,portal 2,"Shooter, Puzzle","Sequel to the acclaimed Portal (2007), Portal ...",Everyone 10+,Высокая,"Soldier Front 2, Meat & Greed, Shadow Wolf Mys..."
3,counter-strike: global offensive,Shooter,Counter-Strike: Global Offensive expands upon ...,Mature,Высокая,"Soldier Front 2, Mobile Suit Gundam Battle Ope..."
4,tomb raider (2013),"Action, Fighting",Нет описания,Mature,Высокая,Нет данных
...,...,...,...,...,...,...
12265,kick ass commandos,"Action, Adventure, Strategy, Arcade, Casual, I...",Нет описания,Все возрастные группы,Низкая,Нет данных
12266,dragon blood,"Action, Adventure, RPG, Massively Multiplayer",Нет описания,Все возрастные группы,Низкая,Нет данных
12267,five: guardians of david,"Action, Adventure, RPG",FIVE: Guardians of David is a Diablo-style act...,Все возрастные группы,Средняя,Нет данных
12268,nyheim,Strategy,Нет описания,Все возрастные группы,Низкая,Нет данных


датасет готов

Для обучения НС будем использовать столбцы Жанры и Целевая аудитория, а Прибыльность будет таргетом Столбцы название игры и описания, очевидно, нет смысла использовать. Можно было бы конкурентов использовать, но данных слишком мало для обучения.

Класс GameIdeaAnalyzer:

Инициализация с указанием пути к файлу данных.

- Метод load_and_preprocess_data загружает данные, обрабатывает их и делит на обучающие и тестовые выборки.

- Метод create_model создает архитектуру нейронной сети.

- Метод train_model обучает модель.

- Метод evaluate_model оценивает качество модели на тестовых данных.

- Метод predict предсказывает результаты на основе входных данных.

- Методы save_model и load_model позволяют сохранять и загружать модель.

Использование класса:

- Создается объект класса GameIdeaAnalyzer.

- Данные загружаются и предобрабатываются.


- Модель создается и обучается.

- Модель оценивается на тестовой выборке.

- Модель сохраняется и загружается для дальнейшего использования.

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model

2024-05-28 17:05:46.104971: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
class GameIdeaAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.model = None
        self.scaler = None
    
    def load_and_preprocess_data(self):
        df = pd.read_csv(self.file_path)
        
        df = df.drop(columns=['Название игры', 'Описание', 'Конкуренты'])
        
        df = pd.get_dummies(df, columns=['Жанры', 'Целевая аудитория', 'Прибыльность'])

        y = df[[col for col in df.columns if col.startswith('Прибыльность_')]]
        X = df.drop(columns=[col for col in df.columns if col.startswith('Прибыльность_')])
        
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X)
        
        X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
        
        return X_train, X_val, X_test, y_train, y_val, y_test
    
    def create_model(self, input_dim, output_dim):
        self.model = Sequential([
            Dense(64, input_dim=input_dim, activation='relu'),
            Dense(32, activation='relu'),
            Dense(output_dim, activation='softmax') 
        ])
        
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    def train_model(self, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))
    
    def evaluate_model(self, X_test, y_test):
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f'Test Accuracy: {accuracy}')
        return accuracy
    
    def evaluate_model_on_val(self, X_val, y_val):
        loss, accuracy = self.model.evaluate(X_val, y_val)
        print(f'Validation Accuracy: {accuracy}')
        return accuracy
    
    def predict(self, X):
        return self.model.predict(X)
    
    
    def save_model(self, model_path):
        self.model.save(model_path)
    
    def load_model(self, model_path):
        self.model = load_model(model_path)


In [9]:
import pandas as pd
df = pd.read_csv('full_games_dataset.csv')
        
df = df.drop(columns=['Название игры', 'Описание', 'Конкуренты'])
        
df = pd.get_dummies(df, columns=['Жанры', 'Целевая аудитория', 'Прибыльность'])

y = df[[col for col in df.columns if col.startswith('Прибыльность_')]]
X = df.drop(columns=[col for col in df.columns if col.startswith('Прибыльность_')])

In [14]:
file_path = 'full_games_dataset.csv'
analyzer = GameIdeaAnalyzer(file_path)

X_train, X_val, X_test, y_train, y_val, y_test = analyzer.load_and_preprocess_data()

analyzer.create_model(input_dim=X_train.shape[1], output_dim=y_train.shape[1])
analyzer.train_model(X_train, y_train, X_val, y_val, epochs=20, batch_size=32)
test_accuracy = analyzer.evaluate_model(X_test, y_test)
val_accuracy = analyzer.evaluate_model_on_val(X_val, y_val)

analyzer.save_model('game_idea_analyzer_model.h5')


Epoch 1/20


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


269/269 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6216 - loss: 0.9387 - val_accuracy: 0.6473 - val_loss: 0.8393
Epoch 2/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6802 - loss: 0.7450 - val_accuracy: 0.6467 - val_loss: 0.8485
Epoch 3/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6864 - loss: 0.7224 - val_accuracy: 0.6348 - val_loss: 0.8714
Epoch 4/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6923 - loss: 0.7174 - val_accuracy: 0.6413 - val_loss: 0.8754
Epoch 5/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6995 - loss: 0.6930 - val_accuracy: 0.6467 - val_loss: 0.8716
Epoch 6/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6893 - loss: 0.7029 - val_accuracy: 0.6386 - val_loss: 0.8959
Epoch 7/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6951 - loss: 0.6952 - val_accuracy: 0.6391 - val_loss: 0.9041
Epoch 8/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7003 - loss: 0.6951 - val_accuracy: 0.6321 - val_

Validation Accuracy: 0.623369574546814


задание со звездочокй 2) Расширьте класс нейросети, добавив возможность проводить анализ конкурентов и предсказывать популярность игры на основе анализа их игровых особенностей.

я теоретически это опишу ниже, но практически вряд ли получится это реализовать на моем датасете, тк количество ячеек с информацией о конкурентах очень мало. Соответственно, для анализа конкурентов можно использовать как мой код выше, который генерировал конкурентов на основе имеющихся данных, так и просто добавить столбец Конкуренты в Х и использовать его (при наличии заполненных ячеек с конкурентами для всех игр.)

In [ ]:
# Загрузка модели (для примера)
analyzer.load_model('game_idea_analyzer_model.h5')

Но вообще так как для табличных данных не рекомендуется использовать нейросети, я предложу простой алгоритм машинного обучения а именно градиентный бустинг для того, чтобы сравнить результаты

In [1]:
import pandas as pd
df = pd.read_csv('full_games_dataset.csv')
        
df = df.drop(columns=['Название игры', 'Описание', 'Конкуренты'])
        
df = pd.get_dummies(df, columns=['Жанры', 'Целевая аудитория', 'Прибыльность'])

y = df[[col for col in df.columns if col.startswith('Прибыльность_')]]
X = df.drop(columns=[col for col in df.columns if col.startswith('Прибыльность_')])

In [3]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 


y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

params = {
    'objective': 'multi:softmax',  
    'num_class': 3,  
    'eval_metric': 'mlogloss',  
    'max_depth': 6,  
    'learning_rate': 0.05,  
    'n_estimators': 20,
}

model = xgb.XGBClassifier(**params)


param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [20, 40, 60]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')


grid_search.fit(X_train, y_train_labels, eval_set=[(X_test, y_test_labels)], verbose=False)


best_params = grid_search.best_params_
print("Лучшие гиперпараметры:", best_params)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test_labels, y_pred)
print("Accuracy на тестовой выборке:", accuracy)


Лучшие гиперпараметры: {'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 40}
Accuracy на тестовой выборке: 0.6515892420537898


результат примерно совпал с результатом НС